# instagram 크롤링

In [1]:
# num_scrolldown = '페이지 스크롤 횟수'
# 예시) num_scrolldown = 2
num_scrolldown = 2

# num_page = '크롤링할 게시글 수'
# 예시) num_page = 6
num_page = 6

In [2]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime

In [3]:
# 로그인 페이지 접속
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)

# 창 최대화
driver.maximize_window()
time.sleep(1)

# 로그인 정보
username = 'gyeonggi_korea'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
element_id.send_keys(username)
element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
element_password.send_keys(password)
time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()
time.sleep(3)

<ipython-input-3-d49f7a4f36a8>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
<ipython-input-3-d49f7a4f36a8>:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
<ipython-input-3-d49f7a4f36a8>:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()


In [4]:
# 크롤링할 페이지 접속
url = 'https://www.instagram.com/gyeonggi_korea/'
# webdriver 로드
driver.get(url)
driver.implicitly_wait(5)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title.text

'gyeonggi_korea(경기도)(@gyeonggi_korea) • Instagram 사진 및 동영상'

In [5]:
# 팔로워 수 수집
raw_num_follower = soup.find_all('span', 'g47SY')
num_follower = int(raw_num_follower[1].text.replace(',',''))
num_follower

9764

In [6]:
# 각 게시글 링크 크롤링
raw_post_link_list = []
for i in range(num_scrolldown):
    # 현재 페이지 크롤링
    driver.implicitly_wait(3)
    soup = bs(driver.page_source, 'lxml')

    # 게시물 링크 주소 수집
    row_post = soup.find_all('div', "Nnq7C weEfm")

    # 게시글 링크 3개씩 가져오기
    for row in row_post:
        # 한줄(row_post)에 3개의 게시글(a 태그) 존재
        a_tags = row.find_all('a')
        for one in a_tags:
            link = one.attrs['href']
            raw_post_link_list.append("https://www.instagram.com" + link)

    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

# 수집한 링크 중 중복 링크 제거
post_link_list = []
for link in raw_post_link_list:
    if link not in post_link_list:
        post_link_list.append(link)

# 수집한 링크 개수 확인
print("수집한 링크의 개수 : ", len(post_link_list))
print("수집한 링크 주소")
print(post_link_list)

수집한 링크의 개수 :  12
수집한 링크 주소
['https://www.instagram.com/p/CbPIO4bPUQJ/', 'https://www.instagram.com/p/CbHbw-uB5B3/', 'https://www.instagram.com/p/Ca_cN1HPMlv/', 'https://www.instagram.com/p/Ca6eIqfPoTV/', 'https://www.instagram.com/p/Ca1mOngPNr7/', 'https://www.instagram.com/p/CarTSR8hpjk/', 'https://www.instagram.com/p/CamCGL3LSHX/', 'https://www.instagram.com/p/Cag862TrWuX/', 'https://www.instagram.com/p/CaWWan2BoZc/', 'https://www.instagram.com/p/CaRgX_svAjY/', 'https://www.instagram.com/p/CaG5pIbv7MV/', 'https://www.instagram.com/p/CZ9rGIHP423/']


In [7]:
date_list = []
title_list = []
num_view_list = []
num_like_list = []
num_comment_list = []
num_share_list = []
num_follower_list = []
crawled_link_list = []

for link in post_link_list[:num_page]:
    # 게시물 타이틀 수집
    driver.get(link)
    time.sleep(3)
    driver.implicitly_wait(3)
    soup = bs(driver.page_source, 'lxml')
    title = soup.title.text[34:]

    # 날짜 수집
    date = soup.find('time').get('datetime')[:10]

    # 인사이트 보기 버튼 클릭
    time.sleep( random.uniform(1,3) )
    section = driver.find_element_by_css_selector('section.fmD5w')
    button = section.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF')
    button.click()

    driver.implicitly_wait(3) # 페이지 로딩 대기 시간
    time.sleep( random.uniform(1,3) ) # 크롤링 차단 방지 랜덤하게 시간 기다리기

    # 조회수
    temp = driver.find_element_by_xpath('/html/body/div[8]/div/div/div/div[2]/div/div/div[2]/div/div/div[7]/div/div/div/div[3]/span[2]')
    if ',' in temp.text:
        temp = temp.text.replace(',','')
        num_view = int(temp)
    else:
        num_view = int(temp.text)

    # 좋아요, 댓글, 공유 수 정보
    temp = driver.find_elements_by_xpath('/html/body/div[8]/div/div/div/div[2]/div/div/div[2]/div/div/div[3]/div/div/div/div/div')

    # 좋아요 수
    if ',' in temp[0].text:
        num_like = int(temp[0].text.replace(',', ''))
    else:
        num_like = int(temp[0].text)
    # 댓글 수
    if ',' in temp[1].text:
        num_comment = int(temp[1].text.replace(',', ''))
    else:
        num_comment = int(temp[1].text)
    # 공유 수
    if ',' in temp[2].text:
        num_share = int(temp[2].text.replace(',', ''))
    else:
        num_share = int(temp[2].text)

    
    # 정보를 리스트에 저장
    date_list.append(date)
    title_list.append(title)
    num_view_list.append(num_view)
    num_like_list.append(num_like)
    num_comment_list.append(num_comment)
    num_share_list.append(num_share)
    crawled_link_list.append(link)

# 팔로워 수 리스트에 저장
num_follower_list.append(num_follower)
for i in range(num_page-1):
    num_follower_list.append(0)

# chromedriver 종료
driver.quit()

<ipython-input-7-44a0070f17c2>:22: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  section = driver.find_element_by_css_selector('section.fmD5w')
C:\anaconda\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead")
<ipython-input-7-44a0070f17c2>:30: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  temp = driver.find_element_by_xpath('/html/body/div[8]/div/div/div/div[2]/div/div/div[2]/div/div/div[7]/div/div/div/div[3]/span[2]')
<ipython-input-7-44a0070f17c2>:38: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpa

In [8]:
# 수집한 자료 확인
print('날짜 리스트 원소 수 : ', len(date_list))
print('제목 리스트 원소 수 : ', len(title_list))
print('조회수 리스트 원소 수 : ', len(num_view_list))
print('좋아요 수 리스트 원소 수 : ', len(num_like_list))
print('공유 수 리스트 원소 수 : ', len(num_share_list))
print('댓글 수 리스트 원소 수 : ', len(num_comment_list))
print('팔로워 수 리스트 원소 수 : ', len(num_follower_list))
print('수집한 링크 리스트 원소 수 : ', len(crawled_link_list))

날짜 리스트 원소 수 :  6
제목 리스트 원소 수 :  6
조회수 리스트 원소 수 :  6
좋아요 수 리스트 원소 수 :  6
공유 수 리스트 원소 수 :  6
댓글 수 리스트 원소 수 :  6
팔로워 수 리스트 원소 수 :  6
수집한 링크 리스트 원소 수 :  6


In [9]:
# 수집한 데이터를 데이터 셋으로 만들기
df = {'날짜':date_list, '제목':title_list, '조회수':num_view_list, '좋아요_수':num_like_list, '공유수':num_share_list, '댓글수':num_comment_list, '팔로워_수':num_follower_list, '링크':crawled_link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-03-18,Gyeonggi Province is home to the Korea Nationa...,17,8,0,0,9764,https://www.instagram.com/p/CbPIO4bPUQJ/
1,2022-03-15,"Have you heard of Empress Myeongseong, an icon...",189,24,0,0,0,https://www.instagram.com/p/CbHbw-uB5B3/
2,2022-03-12,Moran Station’s hottest restaurant with a stat...,230,26,0,0,0,https://www.instagram.com/p/Ca_cN1HPMlv/
3,2022-03-10,It’s a retro throwback \nNamchondong Mural Vil...,247,35,0,4,0,https://www.instagram.com/p/Ca6eIqfPoTV/
4,2022-03-08,A popular place in Seongnam City of Gyeonggi P...,251,25,0,0,0,https://www.instagram.com/p/Ca1mOngPNr7/
5,2022-03-04,Have you heard? (whispering)😝\nKorea’s first-e...,230,25,0,2,0,https://www.instagram.com/p/CarTSR8hpjk/


In [10]:
# 인덱스 내림차순 정렬
final_data = data.sort_index(ascending=False)
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
5,2022-03-04,Have you heard? (whispering)😝\nKorea’s first-e...,230,25,0,2,0,https://www.instagram.com/p/CarTSR8hpjk/
4,2022-03-08,A popular place in Seongnam City of Gyeonggi P...,251,25,0,0,0,https://www.instagram.com/p/Ca1mOngPNr7/
3,2022-03-10,It’s a retro throwback \nNamchondong Mural Vil...,247,35,0,4,0,https://www.instagram.com/p/Ca6eIqfPoTV/
2,2022-03-12,Moran Station’s hottest restaurant with a stat...,230,26,0,0,0,https://www.instagram.com/p/Ca_cN1HPMlv/
1,2022-03-15,"Have you heard of Empress Myeongseong, an icon...",189,24,0,0,0,https://www.instagram.com/p/CbHbw-uB5B3/
0,2022-03-18,Gyeonggi Province is home to the Korea Nationa...,17,8,0,0,9764,https://www.instagram.com/p/CbPIO4bPUQJ/


In [11]:
# 수집한 데이터 파일로 출력하기
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'instagram_' + file_name + '.xlsx'
path = './crawling_data/instagram/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

instagram_20220318-164919.xlsx 파일 생성 완료


# 코드 마지막